In [1]:
import random
import numpy as np

num_of_arrows = 1
rows, cols = (4,4)
no_of_rounds = 100
num_of_pits = 3
coordinate_val = {}

temporary = 1
for temp_i in range(4):
  for temp_j in range(4):
    coordinate_val[(temp_i, temp_j)] = temporary
    temporary+=1
print(coordinate_val)


def check_range(pos):
  lst = []
  for x in pos:
    i,j=x
    # print("i , j = ", i,j,"\n")
    if j>3 or j<0 or i>3 or i<0:
      None
    else: lst.append(x)
  return lst 
count = 0


class Square:
  
  def __init__(self,g, w, p,a ,r, s, b):
    self.glitter = g
    self.wumpus = w
    self.pit = p
    self.agent = a
    self.reward = r
    self.stench = s
    self.breeze = b
    
    self.traversed = False

  def prnt(self):
    print("obj ",self.stench,self.breeze,self.glitter,self.wumpus,self.pit)
   
  def set_breeze(self, num):
    self.breeze = num
  
  def set_stench(self,num):
    self.stench = num
  def get_percept(self):
    return [self.stench , self.breeze, self.glitter,0]   # last parameter for scream




{(0, 0): 1, (0, 1): 2, (0, 2): 3, (0, 3): 4, (1, 0): 5, (1, 1): 6, (1, 2): 7, (1, 3): 8, (2, 0): 9, (2, 1): 10, (2, 2): 11, (2, 3): 12, (3, 0): 13, (3, 1): 14, (3, 2): 15, (3, 3): 16}


In [2]:
agent_pos = (0,0)   
random_numbers = random.sample(range(2, 17), 5)
# pit_list = random.sample(range(2, 16), num_of_pits) 
pit_list = random_numbers[0:3]
wumpus_pos = random_numbers[3]
gold_pos = random_numbers[4]

# wumpus_pos = random.randrange(2,16)
# while wumpus_pos in pit_list:
#   wumpus_pos = random.randrange(2,16)

# gold_pos = random.randrange(2,16)
# while gold_pos in pit_list and gold_pos == wumpus_pos:
#   gold_pos = random.randrange(2,16)



print(gold_pos)
print(wumpus_pos)
print(pit_list)


def find_stench(pos):
  
  cord_val = list(coordinate_val.keys())[list(coordinate_val.values()).index(pos)]
  key_to_value = []
  # print(cord_val)
  i,j = cord_val
  
  for s in check_range([(i,j+1) , (i,j-1) , (i+1,j) , (i-1,j)]):
    key_to_value.append(coordinate_val[s])
  return key_to_value

  
def find_breeze(pit_list):
  br_list = []
  key_to_value = []
  for pos in pit_list:
    cord_val = list(coordinate_val.keys())[list(coordinate_val.values()).index(pos)]
    print(cord_val)
    i,j = cord_val
    # br_list.append(check_range([(i,j+1) , (i,j-1) , (i+1,j) , (i-1,j)]))
    br_list+=check_range([(i,j+1) , (i,j-1) , (i+1,j) , (i-1,j)])
  for s in br_list:
    key_to_value.append(coordinate_val[s])
  return key_to_value



stench_list = find_stench(wumpus_pos)
breeze_list = find_breeze(pit_list)

print("stench list = ",stench_list)
print("breeze list = ", breeze_list)
# pit_list = [3, 9, 12] 
# wumpus_pos = 14
# gold_pos = 16
# stench_list = [10,13,15]
# breeze_list = [2,4,5,7,8,10,11,13]

super_list = []
temp = 1
agent_pos = (0,0)  

for i in range(4):
  world = []
  for j in range(4):
    if temp in pit_list:
      world.append(Square(0,0,1,0,-50,0,0))
      # print("Pit pos set at : ", i,j)
    elif temp == wumpus_pos:
      world.append(Square(0,1,0,0,-50,0,0))
      # print("wumpus pos set at : ", i,j)
    elif temp == gold_pos:
      world.append(Square(1,0,0,0,50,0,0))
      # print("gold pos set at : ", i,j)
    elif (i,j) == agent_pos:
      world.append(Square(0,0,0,1,-1,0,0))
      # print("agent pos set at : ", i,j)
    else:
      if temp in stench_list:
        if temp in breeze_list:
         world.append(Square(0,0,0,0,-10,1,1))
        else:
          world.append(Square(0,0,0,0,-5,1,0))
      elif temp in breeze_list:
        world.append(Square(0,0,0,0,-5,0,1))
      else: world.append(Square(0,0,0,0,-1,0,0))
      # world.append(Square(0,0,0,0,-1,0,0)) 
    

    temp+=1
  super_list.append(world)

# print(super_list[0][0].agent)

3
16
[6, 7, 5]
(1, 1)
(1, 2)
(1, 0)
stench list =  [15, 12]
breeze list =  [7, 5, 10, 2, 8, 6, 11, 3, 6, 9, 1]


In [3]:
def print_grid():
  for i in range(4):
    print("\t\t\t\t\t\t\t\t", end = "")
    for j in range(4):
      str = ""
      if super_list[i][j].agent:
        str = "#"
      #print(super_list[i][j].prnt())
      if super_list[i][j].wumpus:
        str = "W"       
      if super_list[i][j].pit:
        str = "P"
      if super_list[i][j].glitter:
        str = "G"
      if len(str) == 0:
        str+="-"
      print(str, end = "      ")
    print("\n")



In [4]:
class Agent:
    def __init__(self, pos):
        self.position = pos
        self.pos_moves = []
        self.game_over = False
        self.lost = False
        self.lr = 0.2
        self.exp_rate = 0.1
        self.states = []
        self.no_of_wins = 0
        self.no_of_loss = 0

        self.state_values = {}
        for i in range(4):
            for j in range(4):
                self.state_values[(i, j)] = 0  # set initial value to 0

        
        # self.possible_moves()

    def possible_moves(self):
        i,j = self.position[0], self.position[1]
        self.pos_moves = check_range([(i,j+1) , (i,j-1) , (i+1,j) , (i-1,j)])

    def reset(self):
      self.states = []
      self.position = agent_pos
      # print("game reset")

    def high_reward_move(self):
        choose_list = list(range(0,len(self.pos_moves)))
        a,b = self.pos_moves[0]
        # print("a = ", a, "b = ", b)
        current_reward = self.state_values[(a,b)]
        chosen_position = 0

        if np.random.uniform(0, 1) <= self.exp_rate:
          action = np.random.choice(choose_list)
          return self.pos_moves[action]

        for i in choose_list:
         # 
          x,y = self.pos_moves[i]
          new_reward = self.state_values[(x,y)]     
          # print("new reward = ", new_reward)
          if new_reward>=current_reward:   
            current_reward = new_reward
            # print("new R >= current : ", current_reward)
            chosen_position = i
        # print("chosen from dictionary")
        next_move = self.pos_moves[chosen_position]
        return next_move

game_agent = Agent(agent_pos)

  ### GAME LOOP STARTS HERE ###
count = 0
def main_loop():
  super_list[0][0].agent = 1
  game_agent.states.append((0,0))
  print("---------------------------------------------------------ITERATION :",count+1,"-------------------------------------------------------------------------------------")
  # print("agent position start: ", game_agent.position)
  game_agent.possible_moves()
  # print("game_agent.pos moves start: ",game_agent.pos_moves )
  
  
  while True:
      print_grid()
      print("\t\t*********************************************************************************")
      
      i,j = game_agent.position[0], game_agent.position[1]
      if game_agent.game_over:
        reward = super_list[i][j].reward
        game_agent.state_values[(i,j)] = reward
        
        if game_agent.lost:
          print("\t\t\tLOST",end = "\t\t\t\t\t")
          
        else: print("\t\t\tWON", end = "\t\t\t\t\t")

        print("game reward = ", reward)
        
        
        for s in reversed(game_agent.states):
          reward = game_agent.state_values[s] + game_agent.lr * (reward - game_agent.state_values[s])
          game_agent.state_values[s] = round(reward,3)
        game_agent.reset()
        game_agent.game_over = False

        print("\nQ-Learning Matrix : \n")
        for key,value in game_agent.state_values.items():
          # x,y = key
          if value:
            print(value, end = "   ")
          else: print(value, end = "      ")

          if key[1]%3 == 0 and key[1]!=0:
            print("\n")

        # print(game_agent.state_values)
        break

      else:
        
        super_list[i][j].agent = 0

        game_agent.position = game_agent.high_reward_move()
        game_agent.possible_moves()
        # print("game_agent.pos moves new: ",game_agent.pos_moves)
        x,y = game_agent.position[0], game_agent.position[1]
        super_list[x][y].agent = 1
        temp_1 = super_list[x][y]
        game_agent.states.append((x,y))
        # print("NEW position : ", game_agent.position)     
        
        if (temp_1.agent and temp_1.wumpus) or (temp_1.agent and temp_1.pit):
          game_agent.game_over = True 
          game_agent.lost = True
          game_agent.no_of_loss+=1

        elif (temp_1.agent and temp_1.glitter):
          game_agent.game_over = True 
         
         
          game_agent.no_of_wins+=1






while count < no_of_rounds:
  main_loop()
  count+=1

print("------------------------------")
print("no of wins : ", game_agent.no_of_wins)
print("no of loss : ", game_agent.no_of_loss)
 


      



---------------------------------------------------------ITERATION : 1 -------------------------------------------------------------------------------------
								#      -      G      -      

								P      P      P      -      

								-      -      -      -      

								-      -      -      W      

		*********************************************************************************
								-      -      G      -      

								P      P      P      -      

								-      -      -      -      

								-      -      -      W      

		*********************************************************************************
			LOST					game reward =  -50

Q-Learning Matrix : 

-10.0   0      0      0      

-50.0   0      0      0      

0      0      0      0      

0      0      0      0      

---------------------------------------------------------ITERATION : 2 -------------------------------------------------------------------------------------
								#      -      G      -      

